In [1]:
# set up the python
import cosima_cookbook as cc
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import numpy as np
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs
import cmocean as cm
import cartopy.feature as cft
import cftime
import IPython.display
import sys, os, warnings
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)
import matplotlib.ticker as mticker
from xgcm import Grid

In [2]:
# creat the database
db = 'mom6_hycom1.db'
session = cc.database.create_session(db)

In [3]:
directory_list=['/g/data/x77/hm1221/mom6_simulations/wind-up-whole', 
                '/g/data/x77/hm1221/mom6_simulations/wind-down-whole',
               '/g/data/ik11/outputs/mom6-panan/panant-01-hycom1-v13',
               '/g/data/x77/hm1221/mom6_simulations/hybrid_melt_50'
               ]


In [4]:
# load the data to database
cc.database.build_index(directory_list, session)

Indexing experiment: wind-up-whole
Indexing experiment: wind-down-whole
Indexing experiment: panant-01-hycom1-v13
Indexing experiment: hybrid_melt_50


100%|██████████| 1/1 [00:00<00:00, 197.84it/s]


1

# The data is too large, so we first calculate the average and the save them into the /g/data

In [5]:
# set the time period
start_time = '1991-01-01'
end_time = '1991-12-31'

In [6]:
# load mass transport at u and v direction
# 10% wind up
vmo_up = cc.querying.getvar('wind-up-whole', 'vmo', session).sel(time = slice(start_time,end_time))
umo_up = cc.querying.getvar('wind-up-whole', 'umo', session).sel(time = slice(start_time,end_time))
# 50% melt decrease
vmo_melt = cc.querying.getvar('hybrid_melt_50', 'vmo', session).sel(time = slice(start_time,end_time))
umo_melt = cc.querying.getvar('hybrid_melt_50', 'umo', session).sel(time = slice(start_time,end_time))
# control
session1 = cc.database.create_session()
vmo_con = cc.querying.getvar(expt='panant-01-hycom1-v13', variable='vmo', 
                          session=session1, frequency='1 monthly',
                          start_time='1991-01-31 00:00:00', 
                          end_time='2000-12-31 00:00:00').sel(time = slice(start_time,end_time))

umo_con = cc.querying.getvar(expt='panant-01-hycom1-v13', variable='umo', 
                          session=session1, frequency='1 monthly',
                          start_time='1991-01-31 00:00:00', 
                          end_time='2000-12-31 00:00:00').sel(time = slice(start_time,end_time))

In [18]:
umo_con

<xarray.DataArray 'umo' (time: 12, rho2_l: 79, yh: 845, xq: 3601)>
dask.array<getitem, shape=(12, 79, 845, 3601), dtype=float32, chunksize=(1, 12, 121, 515), chunktype=numpy.ndarray>
Coordinates:
  * yh       (yh) float64 -81.11 -81.07 -81.02 -80.98 ... -37.3 -37.22 -37.14
  * rho2_l   (rho2_l) float64 1.016e+03 1.033e+03 ... 1.037e+03 1.038e+03
  * time     (time) object 1991-01-16 12:00:00 ... 1991-12-16 12:00:00
  * xq       (xq) float64 -280.0 -279.9 -279.8 -279.7 ... 79.7 79.8 79.9 80.0
Attributes:
    units:          kg s-1
    long_name:      Ocean Mass X Transport
    cell_methods:   rho2_l:sum yh:sum xq:point time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  ocean_mass_x_transport
    interp_method:  none
    ncfiles:        ['/g/data/ik11/outputs/mom6-panan/panant-01-hycom1-v13/ou...

In [7]:
# interpolate the vmo and umo
ds_con = xr.merge([umo_con[:,:,:,1:], vmo_con[:,:,1:,]])
ds_con.coords['xh'].attrs.update(axis='X')
ds_con.coords['xq'].attrs.update(axis='X', c_grid_axis_shift=0.5)
ds_con.coords['yh'].attrs.update(axis='Y')
ds_con.coords['yq'].attrs.update(axis='Y', c_grid_axis_shift=0.5)

In [8]:
grid = Grid(ds_con, periodic=['X'])
u_con = grid.interp(ds_con.umo, 'X')
v_con = grid.interp(ds_con.vmo, 'Y')
transport_con = ((u_con**2 + v_con**2)**(0.5)).load()

In [9]:
ds_up = xr.merge([umo_up[:,:,:,1:], vmo_up[:,:,1:,]])
ds_up.coords['xh'].attrs.update(axis='X')
ds_up.coords['xq'].attrs.update(axis='X', c_grid_axis_shift=0.5)
ds_up.coords['yh'].attrs.update(axis='Y')
ds_up.coords['yq'].attrs.update(axis='Y', c_grid_axis_shift=0.5)
grid = Grid(ds_up, periodic=['X'])
u_up = grid.interp(ds_up.umo, 'X')
v_up = grid.interp(ds_up.vmo, 'Y')
transport_up = ((u_up**2 + v_up**2)**(0.5)).load()

In [10]:
ds_melt = xr.merge([umo_melt[:,:,:,1:], vmo_melt[:,:,1:,]])
ds_melt.coords['xh'].attrs.update(axis='X')
ds_melt.coords['xq'].attrs.update(axis='X', c_grid_axis_shift=0.5)
ds_melt.coords['yh'].attrs.update(axis='Y')
ds_melt.coords['yq'].attrs.update(axis='Y', c_grid_axis_shift=0.5)
grid = Grid(ds_melt, periodic=['X'])
u_melt = grid.interp(ds_melt.umo, 'X')
v_melt = grid.interp(ds_melt.vmo, 'Y')
transport_melt = ((u_melt**2 + v_melt**2)**(0.5)).load()

In [17]:
# Make all of the northward transport as negative, southward as positive
transport_up

<xarray.DataArray (time: 12, rho2_l: 79, yh: 845, xh: 3600)>
array([[[[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         ...,
         [1.95118047e+04, 2.21535156e+04, 2.46271152e+04, ...,
          1.02598181e+03, 5.75039941e+03, 1.34222637e+04],
         [2.84870215e+04, 3.12098516e+04, 3.24731250e+04, ...,
          8.52562012e+03, 1.67037188e+04, 2.29206816e+04],
         [3.67215938e+04, 4.29369844e+04, 4.74781797e+04, ...,
          1.13698760e+04, 1.88516953e+04, 2.75207461e+04]],

        [[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
...
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

        [[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]]],
      dtype=float32)
Coordinates:
  * yh       (yh) float64 -81.11 -81.07 -81.02 -80.98 ... -37.3 -37.22 -37.14
  * rho2_l   (rho2_l) float64 1.016e+03 1.033e+03 ... 1.037e+03 1.038e+03
  * time     (time) object 1991-01-16 12:00:00 ... 1991-12-16 12:00:00
  * xh       (xh) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95

In [15]:
(np.zeros((2,3))+1)/(np.zeros((2,3))+2)

array([[0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5]])

In [12]:
v_con_direction.values

  return func(*(_execute_task(a, cache) for a in args))



array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [ 1.,  1.,  1., ..., -1.,  1.,  1.],
         [ 1.,  1.,  1., ..., -1.,  1.,  1.],
         [ 1.,  1.,  1., ..., -1.,  1.,  1.]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [ 1.,  1.,  1., ...,  1.,  1.,  1.],
         [ 1.,  1.,  1., ...,  1.,  1.,  1.],
         [ 1.,  1.,  1., ...,  1.,  1.,  1.]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [ 1.,  1., -1., ...,  1.,  1.,  1.],
         [ 1., -1., -1., ...,  1.,  1.,  1.],
         [ 1., -1., -1., ...,  1.,  1.,  1.]],

        ...,

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan

In [12]:
v_con_direction

<xarray.DataArray 'vmo' (time: 12, rho2_l: 79, yh: 845, xh: 3600)>
dask.array<truediv, shape=(12, 79, 845, 3600), dtype=float32, chunksize=(1, 12, 121, 515), chunktype=numpy.ndarray>
Coordinates:
  * yh       (yh) float64 -81.11 -81.07 -81.02 -80.98 ... -37.3 -37.22 -37.14
  * rho2_l   (rho2_l) float64 1.016e+03 1.033e+03 ... 1.037e+03 1.038e+03
  * time     (time) object 1991-01-16 12:00:00 ... 1991-12-16 12:00:00
  * xh       (xh) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95

In [ ]:
# give the transport direction
transport_melt1 = (transport_melt*v_melt_direction).load()`
transport_up1 = (transport_up*v_up_direction).load()
transport_con1 = (transport_con*v_con_direction).load()

  return func(*(_execute_task(a, cache) for a in args))



In [ ]:
transport_melt1 

In [ ]:
# convert to nc files
# set the path
outpath = '/scratch/x77/hm1221/mom6/nc_file'

ds_con = xr.Dataset({'Total_tranport': transport_con1})
ds_con.to_netcdf(outpath+'/transport_con_1991.nc')

ds_up = xr.Dataset({'Total_tranport': transport_up1})
ds_up.to_netcdf(outpath+'/transport_up_1991.nc')

ds_melt = xr.Dataset({'Total_tranport': transport_melt1})
ds_melt.to_netcdf(outpath+'/transport_melt_1991.nc')

# Mask the 1000m isobath contour